In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.outliers_influence import variance_inflation_factor
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Set style for all visualizations
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
# Path variable
data_path = "./capstone_dataset/"

In [ ]:
# Load datasets
obesity_df = pd.read_csv(data_path + "Obesity_rate_US.csv")
poverty_df = pd.read_csv(data_path + "poverty_rate_US.csv")
income_df = pd.read_csv(data_path + "median_income_US.csv")
education_df = pd.read_csv(data_path + "Bachelors_or_higher_US.csv")
food_access_df = pd.read_csv(data_path + "low_income_low_food_access_US.csv")
diabetes_df = pd.read_csv(data_path + "diabetes_rate_US.csv")
inactivity_df = pd.read_csv(data_path + "inactivity_rate_US.csv")

In [ ]:
# Extract county FIPS from census tract GeoID (first 5 digits)
food_access_df['county_fips'] = food_access_df['GeoID'].astype(str).str[:5]

In [ ]:
# Calculate percentage of census tracts classified as "Low Income and Low Access"
food_desert_by_county = food_access_df.groupby('county_fips').apply(
    lambda x: (x['Low Income, Low Access'] == 'Low Income and Low Access').sum() / len(x) * 100
).reset_index(name='food_desert_prevalence')

print(f"Calculated food desert prevalence for {len(food_desert_by_county)} counties")

In [ ]:
# Clean and Standardize County-Level Data
def clean_county_data(df, metric_column):
    df_clean = df[df['Geography Type Description'] == 'County'].copy()
    df_clean['county_fips'] = df_clean['GeoID'].astype(str).str.replace('="', '').str.replace('"', '')
    df_clean['county_fips'] = df_clean['county_fips'].str.zfill(5)  # Ensure 5 digits

    # Keep only FIPS code and the metric
    df_clean = df_clean[['county_fips', metric_column]].copy()
    df_clean[metric_column] = pd.to_numeric(df_clean[metric_column], errors='coerce')
    df_clean = df_clean.dropna(subset=[metric_column])

    return df_clean

In [ ]:
# Identify metric columns for each dataset
obesity_clean = clean_county_data(obesity_df, 'Obesity')
poverty_clean = clean_county_data(poverty_df, 'Percent of People in Poverty')
income_clean = clean_county_data(income_df, 'Median Household Income')
education_clean = clean_county_data(education_df, 'Percent Population with At Least Bachelor\'s Degree')
diabetes_clean = clean_county_data(diabetes_df, 'Diabetes')
inactivity_clean = clean_county_data(inactivity_df, 'Physical Inactivity')

In [ ]:
#  Merge All Data into Master Dataset

# Start with food desert prevalence
master_df = food_desert_by_county.copy()

# Merge each dataset
master_df = master_df.merge(obesity_clean, on='county_fips', how='inner')
master_df = master_df.merge(poverty_clean, on='county_fips', how='inner')
master_df = master_df.merge(income_clean, on='county_fips', how='inner')
master_df = master_df.merge(education_clean, on='county_fips', how='inner')
master_df = master_df.merge(diabetes_clean, on='county_fips', how='inner')
master_df = master_df.merge(inactivity_clean, on='county_fips', how='inner')

In [ ]:
# Rename columns 
master_df.rename(columns={
    'Percent of People in Poverty': 'poverty_pct',
    'Median Household Income': 'median_income_usd',
    'Percent Population with At Least Bachelor\'s Degree': 'education_pct',
    'Diabetes': 'diabetes_pct',
    'Physical Inactivity': 'inactivity_pct',
    'Obesity': 'obesity_pct'
}, inplace=True)

print(master_df.describe())

In [ ]:
# Save master dataset
master_df.to_csv(data_path + "master_analysis_dataset.csv", index=False)
print("\nMaster dataset saved to: master_analysis_dataset.csv")

In [ ]:
# create path to save visualizations
import os
viz_path = "//visualizations/"
os.makedirs(viz_path, exist_ok=True)

In [ ]:
# Distribution plots
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('Variable Distributions', fontsize=16, fontweight='bold')

variables = ['food_desert_prevalence', 'obesity_pct', 'poverty_pct', 'median_income_usd', 'education_pct', 'diabetes_pct']
for ax, var in zip(axes.flat, variables):
    ax.hist(master_df[var], bins=50, edgecolor='black', alpha=0.7)
    ax.set_title(var, fontweight='bold')
    ax.set_xlabel('Value')
    ax.set_ylabel('Frequency')

plt.tight_layout()
plt.savefig(f'{viz_path}01_distributions.png', dpi=300, bbox_inches='tight')
print(f"Saved: 01_distributions.png")
plt.close()

In [ ]:
# Correlation heatmap
fig, ax = plt.subplots(figsize=(10, 8))
corr_matrix = master_df[['food_desert_prevalence', 'obesity_pct', 'poverty_pct',
                           'median_income_usd', 'education_pct', 'diabetes_pct', 'inactivity_pct']].corr()
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0,
            square=True, linewidths=1, ax=ax, cbar_kws={"shrink": 0.8})
ax.set_title('Correlation Matrix: Key Variables', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(f'{viz_path}02_correlation_heatmap.png', dpi=300, bbox_inches='tight')
print(f"Saved: 02_correlation_heatmap.png")
plt.close()

In [ ]:
# Bivariate scatterplot: Food Desert vs Obesity
fig, ax = plt.subplots(figsize=(10, 7))
ax.scatter(master_df['food_desert_prevalence'], master_df['obesity_pct'], alpha=0.6, s=50)
ax.set_xlabel('Food Desert Prevalence (%)', fontsize=12, fontweight='bold')
ax.set_ylabel('Obesity Rate (%)', fontsize=12, fontweight='bold')
ax.set_title('Bivariate Relationship: Food Deserts vs Obesity', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(f'{viz_path}03_bivariate_scatter.png', dpi=300, bbox_inches='tight')
print(f"Saved: 03_bivariate_scatter.png")
plt.close()

In [ ]:

# Regression Model 1 - Simple Linear Regression

# Prepare data for regression
X1 = master_df[['food_desert_prevalence']].copy()
X1 = sm.add_constant(X1)

y = master_df['obesity_pct'].copy()

# Fit model
model1 = sm.OLS(y, X1).fit()
print("\n" + model1.summary().as_text())

# Save summary
with open(f'{viz_path}model1_summary.txt', 'w') as f:
    f.write(model1.summary().as_text())

# Model 1 Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Scatterplot with regression line
ax = axes[0]
ax.scatter(master_df['food_desert_prevalence'], master_df['obesity_pct'], alpha=0.6, s=50)
x_line = np.linspace(master_df['food_desert_prevalence'].min(),
                      master_df['food_desert_prevalence'].max(), 100)
y_line = model1.params['const'] + model1.params['food_desert_prevalence'] * x_line
ax.plot(x_line, y_line, 'r-', linewidth=2, label=f'Fit (R² = {model1.rsquared:.3f})')
ax.set_xlabel('Food Desert Prevalence (%)', fontweight='bold')
ax.set_ylabel('Obesity Rate (%)', fontweight='bold')
ax.set_title('Model 1: Simple Linear Regression', fontweight='bold', fontsize=12)
ax.legend()
ax.grid(True, alpha=0.3)

# Residuals
ax = axes[1]
residuals = model1.resid
ax.scatter(model1.fittedvalues, residuals, alpha=0.6, s=50)
ax.axhline(y=0, color='r', linestyle='--', linewidth=2)
ax.set_xlabel('Fitted Values', fontweight='bold')
ax.set_ylabel('Residuals', fontweight='bold')
ax.set_title('Residual Plot - Model 1', fontweight='bold', fontsize=12)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'{viz_path}04_model1_regression.png', dpi=300, bbox_inches='tight')
print(f"\nSaved: 04_model1_regression.png")
plt.close()


In [ ]:
# Regression Model 2 - Multiple Linear Regression

X2 = master_df[['food_desert_prevalence', 'median_income_usd', 'poverty_pct', 'education_pct']].copy()
X2 = sm.add_constant(X2)

model2 = sm.OLS(y, X2).fit()
print("\n" + model2.summary().as_text())

# Save summary
with open(f'{viz_path}model2_summary.txt', 'w') as f:
    f.write(model2.summary().as_text())

# Calculate VIF for multicollinearity check
print("\nVariance Inflation Factors (VIF):")
vif_data = pd.DataFrame()
vif_data["Variable"] = X2.columns[1:]  # Skip constant
vif_data["VIF"] = [variance_inflation_factor(X2.values, i) for i in range(1, X2.shape[1])]
print(vif_data)

# Save VIF
vif_data.to_csv(f'{viz_path}model2_vif.csv', index=False)

# Model 2 Visualization
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Scatterplot: Food Desert vs Obesity (Model 2 perspective)
ax = axes[0, 0]
ax.scatter(master_df['food_desert_prevalence'], master_df['obesity_pct'], alpha=0.6, s=50)
ax.set_xlabel('Food Desert Prevalence (%)', fontweight='bold')
ax.set_ylabel('Obesity Rate (%)', fontweight='bold')
ax.set_title('Model 2: Food Desert Effect (Controlled)', fontweight='bold')
ax.grid(True, alpha=0.3)

# Residuals
ax = axes[0, 1]
residuals_m2 = model2.resid
ax.scatter(model2.fittedvalues, residuals_m2, alpha=0.6, s=50)
ax.axhline(y=0, color='r', linestyle='--', linewidth=2)
ax.set_xlabel('Fitted Values', fontweight='bold')
ax.set_ylabel('Residuals', fontweight='bold')
ax.set_title('Residual Plot - Model 2', fontweight='bold')
ax.grid(True, alpha=0.3)

# Q-Q Plot
ax = axes[1, 0]
stats.probplot(residuals_m2, dist="norm", plot=ax)
ax.set_title('Q-Q Plot - Model 2', fontweight='bold')
ax.grid(True, alpha=0.3)

# Coefficient comparison
ax = axes[1, 1]
coef_names = ['Food Desert', 'Income', 'Poverty', 'Education']
coef_values = model2.params[1:]
colors = ['green' if x > 0 else 'red' for x in coef_values]
ax.barh(coef_names, coef_values, color=colors, alpha=0.7)
ax.set_xlabel('Coefficient Value', fontweight='bold')
ax.set_title('Model 2 Coefficients', fontweight='bold')
ax.axvline(x=0, color='black', linestyle='-', linewidth=0.8)
ax.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig(f'{viz_path}05_model2_regression.png', dpi=300, bbox_inches='tight')
print(f"Saved: 05_model2_regression.png")
plt.close()


In [ ]:
# Regression Model 3 - Moderated Regression with Interaction Effects

# Create interaction terms
master_df['fd_poverty_interaction'] = master_df['food_desert_prevalence'] * master_df['poverty_pct']
master_df['fd_income_interaction'] = master_df['food_desert_prevalence'] * (master_df['median_income_usd'] / 10000)

# Standardize variables for interaction (improves interpretation)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
vars_to_scale = ['food_desert_prevalence', 'poverty_pct', 'median_income_usd', 'education_pct']
master_df[vars_to_scale] = scaler.fit_transform(master_df[vars_to_scale])

# Recreate interactions with scaled variables
master_df['fd_poverty_interaction'] = master_df['food_desert_prevalence'] * master_df['poverty_pct']

X3 = master_df[['food_desert_prevalence', 'median_income_usd', 'poverty_pct',
                 'education_pct', 'fd_poverty_interaction']].copy()
X3 = sm.add_constant(X3)

model3 = sm.OLS(y, X3).fit()
print("\n" + model3.summary().as_text())

# Save summary
with open(f'{viz_path}model3_summary.txt', 'w') as f:
    f.write(model3.summary().as_text())

# Model 3 Visualization - Interaction Effects
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Interaction visualization: Food Desert effect at different poverty levels
ax = axes[0, 0]
poverty_levels = np.percentile(master_df['poverty_pct'], [25, 50, 75])
colors_interaction = ['blue', 'green', 'red']

for poverty_level, color in zip(poverty_levels, colors_interaction):
    poverty_mask = (master_df['poverty_pct'] > poverty_level - 0.2) & (master_df['poverty_pct'] < poverty_level + 0.2)
    if poverty_mask.sum() > 0:
        ax.scatter(master_df[poverty_mask]['food_desert_prevalence'],
                  master_df[poverty_mask]['obesity_pct'],
                  alpha=0.5, label=f'Poverty {poverty_level:.1f}%', color=color, s=40)

ax.set_xlabel('Food Desert Prevalence (Standardized)', fontweight='bold')
ax.set_ylabel('Obesity Rate (%)', fontweight='bold')
ax.set_title('Model 3: Moderation Effect (Poverty)', fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# Residuals
ax = axes[0, 1]
residuals_m3 = model3.resid
ax.scatter(model3.fittedvalues, residuals_m3, alpha=0.6, s=50)
ax.axhline(y=0, color='r', linestyle='--', linewidth=2)
ax.set_xlabel('Fitted Values', fontweight='bold')
ax.set_ylabel('Residuals', fontweight='bold')
ax.set_title('Residual Plot - Model 3', fontweight='bold')
ax.grid(True, alpha=0.3)

# Q-Q Plot
ax = axes[1, 0]
stats.probplot(residuals_m3, dist="norm", plot=ax)
ax.set_title('Q-Q Plot - Model 3', fontweight='bold')
ax.grid(True, alpha=0.3)

# Coefficient comparison including interaction
ax = axes[1, 1]
coef_names_m3 = ['Food Desert', 'Income', 'Poverty', 'Education', 'FD × Poverty']
coef_values_m3 = model3.params[1:]
colors_m3 = ['green' if x > 0 else 'red' for x in coef_values_m3]
ax.barh(coef_names_m3, coef_values_m3, color=colors_m3, alpha=0.7)
ax.set_xlabel('Coefficient Value', fontweight='bold')
ax.set_title('Model 3 Coefficients (with Interaction)', fontweight='bold')
ax.axvline(x=0, color='black', linestyle='-', linewidth=0.8)
ax.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig(f'{viz_path}06_model3_regression.png', dpi=300, bbox_inches='tight')
print(f"Saved: 06_model3_regression.png")
plt.close()
